<a href="https://colab.research.google.com/github/fabianhuertas1992/Prefactibilidad_SUAN/blob/main/Script/An%C3%A1lisis_DF_Manglares.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Acceso a Drive - Librerias

In [4]:
# @title Montar acceso Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# @title Librerias
!pip install rasterio
!pip install geopandas
!pip install shapely
!pip install matplotlib
!pip install unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 3.9 MB/s eta 0:00:00


# DataFrame dats del GeoJson

In [9]:
# @title Ruta Imagen con Archivos GeoJson
carpeta_drive = "/content/drive/MyDrive/Poligonos"

In [10]:
# @title Contar archivos de la carpeta

import os

def contar_archivos(carpeta):

  try:
    archivos = os.listdir(carpeta)
    cantidad_archivos = len(archivos)
    return cantidad_archivos
  except FileNotFoundError:
    print(f"Error: La carpeta '{carpeta}' no existe.")
    return 0

cantidad = contar_archivos(carpeta_drive)
print(f"Hay {cantidad} archivos en la carpeta {carpeta_drive}")

Hay 2076 archivos en la carpeta /content/drive/MyDrive/Poligonos


In [11]:
# @title Extraer datos y crear DataFrame
import os
import json
import pandas as pd
import unicodedata

# Función para normalizar cadenas (quitar tildes y convertir Ñ/ñ a N/n)
def normalizar_texto(texto):
    if isinstance(texto, str):
        texto = texto.replace("Ñ", "N").replace("ñ", "n")
        texto = unicodedata.normalize("NFKD", texto).encode("ASCII", "ignore").decode("utf-8")
    return texto

# Obtener la lista de archivos GeoJSON en la carpeta
archivos_geojson = [f for f in os.listdir(carpeta_drive) if f.endswith(".geojson")]

# Lista para almacenar los datos
datos = []

# Leer cada archivo GeoJSON
for archivo in archivos_geojson:
    ruta_archivo = os.path.join(carpeta_drive, archivo)
    with open(ruta_archivo, "r", encoding="utf-8") as f:
        geojson_data = json.load(f)
        for feature in geojson_data["features"]:
            # Extraer propiedades y normalizarlas
            propiedades = {normalizar_texto(k): normalizar_texto(v) for k, v in feature["properties"].items()}

            # Agregar el nombre del archivo
            propiedades["Archivo"] = normalizar_texto(archivo)

            # Extraer las coordenadas y formatearlas
            if feature["geometry"]["type"] == "Polygon":
                coords = [[lon, lat] for ring in feature["geometry"]["coordinates"] for lon, lat, *_ in ring]
                propiedades["Poligono"] = str(coords)  # Guardar en formato string sin tildes ni ñ

            datos.append(propiedades)

# Convertir a DataFrame
df = pd.DataFrame(datos)

print("✅ Datos extraídos y normalizados.")
df


✅ Datos extraídos y normalizados.


,Depto,Restauraci,zona,Shape_Leng,Shape_Area,Area_ha,Archivo,Poligono
0,Narino,Restauracion Activa,Narino,560.392623,11925.924601,1.192592,Narino_560.392623449.geojson,"[[-78.61743003297217, 2.310687846041306], [-78..."
1,Narino,Restauracion Activa,Narino,1538.249271,47932.706057,4.793271,Narino_1538.24927121.geojson,"[[-78.6355105759848, 2.309385949118805], [-78...."
2,Narino,Restauracion Activa,Narino,676.423474,23141.466229,2.314147,Narino_676.423473557.geojson,"[[-78.62558918685171, 2.310687290403761], [-78..."
3,Narino,Restauracion Activa,Narino,1731.795248,32024.531823,3.202453,Narino_1731.79524804.geojson,"[[-78.61327318296796, 2.320396349695263], [-78..."
4,Narino,Restauracion Activa,Narino,793.963864,10664.562951,1.066456,Narino_793.963864221.geojson,"[[-78.61427941518029, 2.310985757377912], [-78..."
...,...,...,...,...,...,...,...,...
2071,Choco,Restauracion Activa,Zona 2 - Choco (Bajo Baudo),181.725479,691.657591,0.069166,Choco_181.725479264.geojson,"[[-77.3085641942588, 4.809580030392463], [-77...."
2072,Choco,Restauracion Activa,Zona 2 - Choco (Bajo Baudo),589.301863,17880.350574,1.788035,Choco_589.301862777.geojson,"[[-77.2897446030988, 4.807926009288338], [-77...."
2073,Choco,Restauracion Activa,Zona 2 - Choco (Bajo Baudo),51.284200,80.904360,0.008090,Choco_51.2841997378.geojson,"[[-77.30807190167423, 4.808746153048969], [-77..."
2074,Choco,Restauracion Activa,Zona 2 - Choco (Bajo Baudo),10.641806,3.916600,0.000392,Choco_10.6418057464.geojson,"[[-77.3083871092495, 4.80886239956291], [-77.3..."


In [12]:
# @title Crea Coordenadas
import os
import json
import pandas as pd
import unicodedata

# Función para normalizar cadenas (quitar tildes y convertir Ñ/ñ a N/n)
def normalizar_texto(texto):
    if isinstance(texto, str):
        texto = texto.replace("Ñ", "N").replace("ñ", "n")
        texto = unicodedata.normalize("NFKD", texto).encode("ASCII", "ignore").decode("utf-8")
    return texto

# Obtener la lista de archivos GeoJSON en la carpeta
archivos_geojson = [f for f in os.listdir(carpeta_drive) if f.endswith(".geojson")]

# Lista para almacenar los datos
datos = []

# Leer cada archivo GeoJSON
for archivo in archivos_geojson:
    ruta_archivo = os.path.join(carpeta_drive, archivo)
    with open(ruta_archivo, "r", encoding="utf-8") as f:
        geojson_data = json.load(f)
        for feature in geojson_data["features"]:
            # Extraer propiedades y normalizarlas
            propiedades = {normalizar_texto(k): normalizar_texto(v) for k, v in feature["properties"].items()}

            # Asegurar que las columnas originales estén en el DataFrame
            columnas_originales = ["Depto", "Restauraci", "zona", "Shape_Leng", "Shape_Area", "Area_ha"]
            for col in columnas_originales:
                if col not in propiedades:
                    propiedades[col] = None  # Rellenar si falta alguna columna

            # Agregar el nombre del archivo
            propiedades["Archivo"] = normalizar_texto(archivo)

            # Extraer las coordenadas y obtener solo el primer punto del polígono
            if feature["geometry"]["type"] == "Polygon":
                coords = [[lon, lat] for ring in feature["geometry"]["coordinates"] for lon, lat, *_ in ring]
                propiedades["Poligono"] = str(coords)  # Guardar en formato string sin tildes ni ñ


                # Si hay al menos un punto, extraer el primero
                if len(coords) >= 1:
                    lat1, lon1 = coords[0][1], coords[0][0]  # Extraer la primera coordenada
                    propiedades["Coordenadas"] = f"{lat1}, {lon1}"
                    propiedades["Latitud_1"] = lat1
                    propiedades["Longitud_1"] = lon1
                else:
                    propiedades["Coordenadas"] = None
                    propiedades["Latitud_1"] = None
                    propiedades["Longitud_1"] = None

            datos.append(propiedades)

# Convertir a DataFrame
df = pd.DataFrame(datos)
df


,Depto,Restauraci,zona,Shape_Leng,Shape_Area,Area_ha,Archivo,Poligono,Coordenadas,Latitud_1,Longitud_1
0,Narino,Restauracion Activa,Narino,560.392623,11925.924601,1.192592,Narino_560.392623449.geojson,"[[-78.61743003297217, 2.310687846041306], [-78...","2.310687846041306, -78.61743003297217",2.310688,-78.617430
1,Narino,Restauracion Activa,Narino,1538.249271,47932.706057,4.793271,Narino_1538.24927121.geojson,"[[-78.6355105759848, 2.309385949118805], [-78....","2.309385949118805, -78.6355105759848",2.309386,-78.635511
2,Narino,Restauracion Activa,Narino,676.423474,23141.466229,2.314147,Narino_676.423473557.geojson,"[[-78.62558918685171, 2.310687290403761], [-78...","2.310687290403761, -78.62558918685171",2.310687,-78.625589
3,Narino,Restauracion Activa,Narino,1731.795248,32024.531823,3.202453,Narino_1731.79524804.geojson,"[[-78.61327318296796, 2.320396349695263], [-78...","2.320396349695263, -78.61327318296796",2.320396,-78.613273
4,Narino,Restauracion Activa,Narino,793.963864,10664.562951,1.066456,Narino_793.963864221.geojson,"[[-78.61427941518029, 2.310985757377912], [-78...","2.310985757377912, -78.61427941518029",2.310986,-78.614279
...,...,...,...,...,...,...,...,...,...,...,...
2071,Choco,Restauracion Activa,Zona 2 - Choco (Bajo Baudo),181.725479,691.657591,0.069166,Choco_181.725479264.geojson,"[[-77.3085641942588, 4.809580030392463], [-77....","4.809580030392463, -77.3085641942588",4.809580,-77.308564
2072,Choco,Restauracion Activa,Zona 2 - Choco (Bajo Baudo),589.301863,17880.350574,1.788035,Choco_589.301862777.geojson,"[[-77.2897446030988, 4.807926009288338], [-77....","4.807926009288338, -77.2897446030988",4.807926,-77.289745
2073,Choco,Restauracion Activa,Zona 2 - Choco (Bajo Baudo),51.284200,80.904360,0.008090,Choco_51.2841997378.geojson,"[[-77.30807190167423, 4.808746153048969], [-77...","4.808746153048969, -77.30807190167423",4.808746,-77.308072
2074,Choco,Restauracion Activa,Zona 2 - Choco (Bajo Baudo),10.641806,3.916600,0.000392,Choco_10.6418057464.geojson,"[[-77.3083871092495, 4.80886239956291], [-77.3...","4.80886239956291, -77.3083871092495",4.808862,-77.308387


In [13]:
# @title Realiza consultas API del IGAC para traer ID_Catastral y poligono Catastral
import requests
import json
import pandas as pd
from shapely.geometry import shape
from shapely.ops import transform
from pyproj import Transformer

# Definir la función para obtener el polígono y el ID Catastral
def obtener_poligono_por_geometria(lat, lon, radio=10):
    BASE_URL = "https://services2.arcgis.com/RVvWzU3lgJISqdke/ArcGIS/rest/services/CATASTRO_PUBLICO_Junio_30_2024/FeatureServer/"
    tableNumber = 14
    COMMAND = "query"

    query = {
        'geometry': f"{lon},{lat}",  # Lon, Lat
        'geometryType': 'esriGeometryPoint',
        'inSR': '4326',  # Sistema de referencia espacial (WGS 84)
        'spatialRel': 'esriSpatialRelIntersects',
        'distance': radio,  # Radio en metros para buscar alrededor del punto
        'units': 'esriSRUnit_Meter',
        'returnGeometry': 'true',
        'outFields': '*',  # Obtener todos los campos
        'f': 'geojson',
        'resultRecordCount': 1  # Limitar el resultado a un solo registro
    }

    URL = f"{BASE_URL}/{tableNumber}/{COMMAND}"

    response = requests.get(URL, params=query)
    if response.status_code == 200:
        return response.json()
    else:
        return None

# Crear listas para almacenar los nuevos datos
id_catastral_list = []
poligono_list = []
area_ha_list = []

# Iterar sobre cada fila del DataFrame existente
for index, row in df.iterrows():
    lat = row["Latitud_1"]
    lon = row["Longitud_1"]

    try:
        resultado = obtener_poligono_por_geometria(lat, lon)
        if resultado and resultado['features']:
            feature = resultado['features'][0]
            id_catastral = feature['properties'].get('CODIGO', 'ID No disponible')
            poligono = json.dumps(feature['geometry']['coordinates'])  # Convertir a JSON string

            # Transformar a un sistema de referencia en metros (EPSG:3857) para calcular el área
            geom = shape(feature["geometry"])
            transformer = Transformer.from_crs("epsg:4326", "epsg:3857", always_xy=True)
            geom_transformed = transform(transformer.transform, geom)

            area_m2 = geom_transformed.area
            area_ha = area_m2 / 10000  # Convertir de m² a ha
        else:
            id_catastral = "No encontrado"
            poligono = None
            area_ha = None
    except:
        id_catastral = "Error"
        poligono = None
        area_ha = None

    id_catastral_list.append(id_catastral)
    poligono_list.append(poligono)
    area_ha_list.append(area_ha)

# Agregar los nuevos datos al DataFrame
df["ID_Catastral"] = id_catastral_list
df["Polígono_Catastral"] = poligono_list
df["Área_Ha_Catastral"] = area_ha_list

In [14]:
df

,Depto,Restauraci,zona,Shape_Leng,Shape_Area,Area_ha,Archivo,Poligono,Coordenadas,Latitud_1,Longitud_1,ID_Catastral,Polígono_Catastral,Área_Ha_Catastral
0,Narino,Restauracion Activa,Narino,560.392623,11925.924601,1.192592,Narino_560.392623449.geojson,"[[-78.61743003297217, 2.310687846041306], [-78...","2.310687846041306, -78.61743003297217",2.310688,-78.617430,No encontrado,None,NaN
1,Narino,Restauracion Activa,Narino,1538.249271,47932.706057,4.793271,Narino_1538.24927121.geojson,"[[-78.6355105759848, 2.309385949118805], [-78....","2.309385949118805, -78.6355105759848",2.309386,-78.635511,No encontrado,None,NaN
2,Narino,Restauracion Activa,Narino,676.423474,23141.466229,2.314147,Narino_676.423473557.geojson,"[[-78.62558918685171, 2.310687290403761], [-78...","2.310687290403761, -78.62558918685171",2.310687,-78.625589,No encontrado,None,NaN
3,Narino,Restauracion Activa,Narino,1731.795248,32024.531823,3.202453,Narino_1731.79524804.geojson,"[[-78.61327318296796, 2.320396349695263], [-78...","2.320396349695263, -78.61327318296796",2.320396,-78.613273,No encontrado,None,NaN
4,Narino,Restauracion Activa,Narino,793.963864,10664.562951,1.066456,Narino_793.963864221.geojson,"[[-78.61427941518029, 2.310985757377912], [-78...","2.310985757377912, -78.61427941518029",2.310986,-78.614279,No encontrado,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2071,Choco,Restauracion Activa,Zona 2 - Choco (Bajo Baudo),181.725479,691.657591,0.069166,Choco_181.725479264.geojson,"[[-77.3085641942588, 4.809580030392463], [-77....","4.809580030392463, -77.3085641942588",4.809580,-77.308564,No encontrado,None,NaN
2072,Choco,Restauracion Activa,Zona 2 - Choco (Bajo Baudo),589.301863,17880.350574,1.788035,Choco_589.301862777.geojson,"[[-77.2897446030988, 4.807926009288338], [-77....","4.807926009288338, -77.2897446030988",4.807926,-77.289745,No encontrado,None,NaN
2073,Choco,Restauracion Activa,Zona 2 - Choco (Bajo Baudo),51.284200,80.904360,0.008090,Choco_51.2841997378.geojson,"[[-77.30807190167423, 4.808746153048969], [-77...","4.808746153048969, -77.30807190167423",4.808746,-77.308072,No encontrado,None,NaN
2074,Choco,Restauracion Activa,Zona 2 - Choco (Bajo Baudo),10.641806,3.916600,0.000392,Choco_10.6418057464.geojson,"[[-77.3083871092495, 4.80886239956291], [-77.3...","4.80886239956291, -77.3083871092495",4.808862,-77.308387,No encontrado,None,NaN


In [15]:
#  @title Filtrar los diferentes a No encontrado
df_filtered = df[df['ID_Catastral'] != 'No encontrado']
df_filtered

,Depto,Restauraci,zona,Shape_Leng,Shape_Area,Area_ha,Archivo,Poligono,Coordenadas,Latitud_1,Longitud_1,ID_Catastral,Polígono_Catastral,Área_Ha_Catastral
24,Narino,Restauracion Activa,Narino,515.813045,9146.187311,0.914619,Narino_515.813044567.geojson,"[[-78.58621963870664, 2.370188990508232], [-78...","2.370188990508232, -78.58621963870664",2.370189,-78.586220,528350001000001230029000000000,"[[[-78.5297214211279, 2.31122504501179], [-78....",3764.738941
118,Narino,Manejo Forestal Sostenible e Incentivos a la C...,Zona 1 - Narino (Tumaco),1463.287557,34844.481880,3.484448,Narino_1463.28755674.geojson,"[[-78.81216431232743, 1.7415638414944], [-78.8...","1.7415638414944, -78.81216431232743",1.741564,-78.812164,528350001000000860001000000000,"[[[[-78.8375774869721, 1.68822461854656], [-78...",6147.637984
120,Narino,Manejo Forestal Sostenible e Incentivos a la C...,Zona 1 - Narino (Tumaco),320.985667,1917.714312,0.191771,Narino_320.985666669.geojson,"[[-78.81084976063883, 1.745108760943289], [-78...","1.745108760943289, -78.81084976063883",1.745109,-78.810850,528350001000000860001000000000,"[[[[-78.8375774869721, 1.68822461854656], [-78...",6147.637984
123,Narino,Manejo Forestal Sostenible e Incentivos a la C...,Zona 1 - Narino (Tumaco),793.765737,20006.122380,2.000612,Narino_793.765736509.geojson,"[[-78.80784725967246, 1.741156006467768], [-78...","1.741156006467768, -78.80784725967246",1.741156,-78.807847,528350001000000860001000000000,"[[[[-78.8375774869721, 1.68822461854656], [-78...",6147.637984
124,Narino,Manejo Forestal Sostenible e Incentivos a la C...,Zona 1 - Narino (Tumaco),1417.038606,58571.494331,5.857149,Narino_1417.03860566.geojson,"[[-78.81216431232743, 1.7415638414944], [-78.8...","1.7415638414944, -78.81216431232743",1.741564,-78.812164,528350001000000860001000000000,"[[[[-78.8375774869721, 1.68822461854656], [-78...",6147.637984
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2053,Choco,Manejo Forestal Sostenible e Incentivos a la C...,Zona 2 - Choco (Bajo Baudo),232.090255,2904.309082,0.290431,Choco_232.090254866.geojson,"[[-77.29248434937276, 4.849195734549609], [-77...","4.849195734549609, -77.29248434937276",4.849196,-77.292484,270770000000000000300000000000,"[[[-77.2225456280782, 4.87778250519826], [-77....",18271.541309
2056,Choco,Manejo Forestal Sostenible e Incentivos a la C...,Zona 2 - Choco (Bajo Baudo),472.264130,7792.707626,0.779271,Choco_472.264129663.geojson,"[[-77.31988296427706, 4.848117756966969], [-77...","4.848117756966969, -77.31988296427706",4.848118,-77.319883,270770000000000000300000000000,"[[[-77.2225456280782, 4.87778250519826], [-77....",18271.541309
2060,Choco,Rehabilitacion Funcional,Zona 2 - Choco (Bajo Baudo),2720.964611,71809.942210,7.180994,Choco_2720.9646114.geojson,"[[-77.34027472349183, 4.855304278369998], [-77...","4.855304278369998, -77.34027472349183",4.855304,-77.340275,270770000000000000300000000000,"[[[-77.2225456280782, 4.87778250519826], [-77....",18271.541309
2063,Choco,Rehabilitacion Funcional,Zona 2 - Choco (Bajo Baudo),2084.155494,46507.022964,4.650702,Choco_2084.15549421.geojson,"[[-77.34027472349183, 4.855304278369998], [-77...","4.855304278369998, -77.34027472349183",4.855304,-77.340275,270770000000000000300000000000,"[[[-77.2225456280782, 4.87778250519826], [-77....",18271.541309


# **Crear Url de los archivos en la carpeta**

In [18]:
# @title Crea copia del DF
dfurl = df.copy()


In [19]:
# @title Autenticación ingreso a Drive
from google.colab import auth
from googleapiclient.discovery import build

# Autenticar en Google Drive
auth.authenticate_user()
drive_service = build('drive', 'v3')

# ID de la carpeta en Google Drive (AJUSTA ESTO)
folder_id = "1-0VK3F5pw9naWIGlNiVd1VidEjpaZIWO"

In [20]:
# @title Lista de todos los archivos
archivos_encontrados = []
page_token = None

while True:
    query = f"'{folder_id}' in parents"
    results = drive_service.files().list(
        q=query,
        fields="nextPageToken, files(id, name, mimeType)",
        pageSize=1000,  # Solicita hasta 1000 archivos por página
        pageToken=page_token  # Usa la paginación para obtener más archivos
    ).execute()

    archivos_encontrados.extend(results.get("files", []))  # Agrega los archivos encontrados

    page_token = results.get("nextPageToken")  # Obtiene el siguiente token de página
    if not page_token:
        break  # Si no hay más páginas, salimos del bucle

# Mostrar cuántos archivos se encontraron
print(f"✅ Se encontraron {len(archivos_encontrados)} archivos en la carpeta.")

# Mostrar una lista de los primeros 20 archivos encontrados
for archivo in archivos_encontrados[:20]:  # Mostrar solo los primeros 20 para no sobrecargar la salida
    print(f"📂 Nombre: {archivo['name']} | 📌 ID: {archivo['id']} | 🏷️ Tipo MIME: {archivo['mimeType']}")

# Opción: Convertir a DataFrame si necesitas analizar la información
import pandas as pd
df_archivos = pd.DataFrame(archivos_encontrados)



✅ Se encontraron 2076 archivos en la carpeta.
📂 Nombre: Choco_372.805928501.geojson | 📌 ID: 1BgavYjp90A0pex7anGAVLKv77IBoV6Eg | 🏷️ Tipo MIME: application/octet-stream
📂 Nombre: Choco_1062.4753969.geojson | 📌 ID: 1Bh-mYs--xACodJ0YY_oH1FRtJIPESTMF | 🏷️ Tipo MIME: application/octet-stream
📂 Nombre: Choco_613.681155072.geojson | 📌 ID: 1Bh9uNPcVpXacJwIK7gMfhM7im3ftYnSD | 🏷️ Tipo MIME: application/octet-stream
📂 Nombre: Choco_378.472223508.geojson | 📌 ID: 1BiHQow6_uQ4i1tKWSlXUuxVbWHWhpYDY | 🏷️ Tipo MIME: application/octet-stream
📂 Nombre: Choco_273.96723845.geojson | 📌 ID: 1BiuF_eMCHvRrQf1z_PqaVmThvp-qg6l4 | 🏷️ Tipo MIME: application/octet-stream
📂 Nombre: Choco_324.5812217.geojson | 📌 ID: 1Bj1HAVa87uwKJV2vpO6ELTsjlwvCfpO6 | 🏷️ Tipo MIME: application/octet-stream
📂 Nombre: Choco_78.2907549054.geojson | 📌 ID: 1Bj6yZp__ahTbWr4kZ0h6TyeMFvRCx61Y | 🏷️ Tipo MIME: application/octet-stream
📂 Nombre: Choco_4068.12534962.geojson | 📌 ID: 1BjUr_i2ZxMvcZZLH6shtWmdxknAhGGMd | 🏷️ Tipo MIME: application/oct

In [21]:
df_archivos

,mimeType,id,name
0,application/octet-stream,1BgavYjp90A0pex7anGAVLKv77IBoV6Eg,Choco_372.805928501.geojson
1,application/octet-stream,1Bh-mYs--xACodJ0YY_oH1FRtJIPESTMF,Choco_1062.4753969.geojson
2,application/octet-stream,1Bh9uNPcVpXacJwIK7gMfhM7im3ftYnSD,Choco_613.681155072.geojson
3,application/octet-stream,1BiHQow6_uQ4i1tKWSlXUuxVbWHWhpYDY,Choco_378.472223508.geojson
4,application/octet-stream,1BiuF_eMCHvRrQf1z_PqaVmThvp-qg6l4,Choco_273.96723845.geojson
...,...,...,...
2071,application/octet-stream,1-d6Gx_48l_gdMJMoyp69-I8GX5mI0sEc,Valle_del_Cauca_69.1049732829.geojson
2072,application/octet-stream,1-kvgpI33i-eZX5HyHe1HqpdqnMlanqGm,Valle_del_Cauca_743.565769903.geojson
2073,application/octet-stream,1-lSivKsxDM88w-aXiRSE5zOVyHZcF5mK,Valle_del_Cauca_731.372820599.geojson
2074,application/octet-stream,1-nVJO60MCjk912FoP9As6jmZ3xpn1dUo,Valle_del_Cauca_425.736134204.geojson


In [22]:
# @title Crae las url por cada archivo
# Crear un diccionario con el nombre del archivo y su URL pública
archivos_drive = {}
for archivo in archivos_encontrados:
    file_name = archivo["name"]

    # Filtrar solo los archivos que terminan en .geojson
    if file_name.endswith(".geojson"):
        file_id = archivo["id"]
        file_url = f"https://drive.google.com/file/d/{file_id}/view?usp=sharing"  # URL accesible desde el navegador
        archivos_drive[file_name] = file_url

# Verificar si se encontraron archivos .geojson
if archivos_drive:
    print(f"✅ Se generaron {len(archivos_drive)} URLs de archivos GeoJSON correctamente.")
else:
    print("⚠️ No se encontraron archivos GeoJSON en la carpeta.")

# Agregar la URL de cada archivo GeoJSON al DataFrame
dfurl["URL_GeoJSON"] = dfurl["Archivo"].map(archivos_drive)




✅ Se generaron 2076 URLs de archivos GeoJSON correctamente.


In [23]:
dfurl

,Depto,Restauraci,zona,Shape_Leng,Shape_Area,Area_ha,Archivo,Poligono,Coordenadas,Latitud_1,Longitud_1,ID_Catastral,Polígono_Catastral,Área_Ha_Catastral,URL_GeoJSON
0,Narino,Restauracion Activa,Narino,560.392623,11925.924601,1.192592,Narino_560.392623449.geojson,"[[-78.61743003297217, 2.310687846041306], [-78...","2.310687846041306, -78.61743003297217",2.310688,-78.617430,No encontrado,None,NaN,https://drive.google.com/file/d/1-7rbep9eB6cIK...
1,Narino,Restauracion Activa,Narino,1538.249271,47932.706057,4.793271,Narino_1538.24927121.geojson,"[[-78.6355105759848, 2.309385949118805], [-78....","2.309385949118805, -78.6355105759848",2.309386,-78.635511,No encontrado,None,NaN,https://drive.google.com/file/d/1-7ReAUSf1zW4j...
2,Narino,Restauracion Activa,Narino,676.423474,23141.466229,2.314147,Narino_676.423473557.geojson,"[[-78.62558918685171, 2.310687290403761], [-78...","2.310687290403761, -78.62558918685171",2.310687,-78.625589,No encontrado,None,NaN,https://drive.google.com/file/d/1-7QgYTOP58wqa...
3,Narino,Restauracion Activa,Narino,1731.795248,32024.531823,3.202453,Narino_1731.79524804.geojson,"[[-78.61327318296796, 2.320396349695263], [-78...","2.320396349695263, -78.61327318296796",2.320396,-78.613273,No encontrado,None,NaN,https://drive.google.com/file/d/1-7EhPeOSD85eN...
4,Narino,Restauracion Activa,Narino,793.963864,10664.562951,1.066456,Narino_793.963864221.geojson,"[[-78.61427941518029, 2.310985757377912], [-78...","2.310985757377912, -78.61427941518029",2.310986,-78.614279,No encontrado,None,NaN,https://drive.google.com/file/d/1-6ruUGk7Ib_gU...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2071,Choco,Restauracion Activa,Zona 2 - Choco (Bajo Baudo),181.725479,691.657591,0.069166,Choco_181.725479264.geojson,"[[-77.3085641942588, 4.809580030392463], [-77....","4.809580030392463, -77.3085641942588",4.809580,-77.308564,No encontrado,None,NaN,https://drive.google.com/file/d/13g46YIcQEpldK...
2072,Choco,Restauracion Activa,Zona 2 - Choco (Bajo Baudo),589.301863,17880.350574,1.788035,Choco_589.301862777.geojson,"[[-77.2897446030988, 4.807926009288338], [-77....","4.807926009288338, -77.2897446030988",4.807926,-77.289745,No encontrado,None,NaN,https://drive.google.com/file/d/13fHBbvMD-_Is8...
2073,Choco,Restauracion Activa,Zona 2 - Choco (Bajo Baudo),51.284200,80.904360,0.008090,Choco_51.2841997378.geojson,"[[-77.30807190167423, 4.808746153048969], [-77...","4.808746153048969, -77.30807190167423",4.808746,-77.308072,No encontrado,None,NaN,https://drive.google.com/file/d/13cmn5LrOndgyE...
2074,Choco,Restauracion Activa,Zona 2 - Choco (Bajo Baudo),10.641806,3.916600,0.000392,Choco_10.6418057464.geojson,"[[-77.3083871092495, 4.80886239956291], [-77.3...","4.80886239956291, -77.3083871092495",4.808862,-77.308387,No encontrado,None,NaN,https://drive.google.com/file/d/13aTU877YHGF9Y...


# Análisis de Imágenes

In [25]:
# @title Rutas de las carpetas con imágenes anteriores y posteriores
carpeta_anterior = "/content/drive/MyDrive/poligonos_imagenes/choco_anterior"
carpeta_posterior = "/content/drive/MyDrive/poligonos_imagenes/choco_posterior"

In [26]:
# @title numero total de archivos de cada carpeta

import os

def count_files_in_folders(folder_paths):

  file_counts = {}
  for folder_path in folder_paths:
    if os.path.exists(folder_path):
      file_count = 0
      for _, _, files in os.walk(folder_path):
        file_count += len(files)
      file_counts[folder_path] = file_count
    else:
      file_counts[folder_path] = "Folder not found"
  return file_counts



folder_list = [carpeta_anterior, carpeta_posterior]
file_counts = count_files_in_folders(folder_list)

for folder_path, count in file_counts.items():
  print(f"Folder: {folder_path}, File count: {count}")


Folder: /content/drive/MyDrive/poligonos_imagenes/choco_anterior, File count: 1240
Folder: /content/drive/MyDrive/poligonos_imagenes/choco_posterior, File count: 1240


In [8]:
# @title DataFrame Comparativo Anterior y Posterior con datos
# Importar librerías necesarias
import os
import re
import numpy as np
import rasterio
import pandas as pd
import unicodedata

# 📌 Tamaño del píxel en metros
PIXEL_SIZE = 10
HECTARE_CONVERSION = 10000

# 📌 Función para normalizar texto
def normalizar_texto(texto):
    return texto.replace("Ñ", "N").replace("ñ", "n").lower()

# 📌 Función para agrupar imágenes en cada carpeta
def agrupar_imagenes(carpeta):
    archivos = [f for f in os.listdir(carpeta) if f.endswith(".tiff")]
    patrones = {}

    # 📌 Nueva expresión regular para la estructura actual
    regex = re.compile(r"^(.+?)_(\d+\.\d+?)_(\d{4}-\d{2}-\d{2})_(.+?)_sentinel-2_(B\d+)\.tiff$")
  #patron = re.compile(r'^(.+?)_(\d+\.\d+)_(\d{4}-\d{2}-\d{2})_([^_]+?)(_sentinel-2_B\d+\.tiff)$')
    for archivo in archivos:
        match = regex.match(archivo)

        if match:
            depto, shape_leng, fecha, zona, banda = match.groups()
            clave = f"{normalizar_texto(depto)}_{normalizar_texto(shape_leng)}_{normalizar_texto(zona)}"
            banda = banda.replace("B011", "B11")  # Corrección opcional de bandas

            if clave not in patrones:
                patrones[clave] = {}

            patrones[clave][banda] = os.path.join(carpeta, archivo)

        else:
            # 📌 Depuración: Mostrar qué archivo no coincide
            print(f"⚠️ No coincide: {archivo}")

            # Intentar una coincidencia parcial para ver qué partes se extraen
            test_match = re.match(r"^(.+?)_(\d+\.\d+?)_(.+?)_(.+?)_sentinel-2_(B\d+)\.tiff$", archivo)
            if test_match:
                print(f"🔎 Coincide parcialmente: {test_match.groups()}")
            else:
                print(f"❌ No se encontró ninguna coincidencia en absoluto para: {archivo}")

    return patrones

# 📌 Función para calcular NDVI
def calcular_ndvi(banda_NIR, banda_roja):
    suma = banda_NIR + banda_roja
    suma[suma == 0] = np.nan  # Evita división por cero
    return np.nan_to_num((banda_NIR - banda_roja) / suma)

# 📌 Función para calcular NDWI
def calcular_ndwi(banda_verde, banda_SWIR):
    suma = banda_verde + banda_SWIR
    suma[suma == 0] = np.nan
    return np.nan_to_num((banda_verde - banda_SWIR) / suma)

# 📌 Función para clasificar cobertura de manglar y agua
def clasificar_cobertura(ndvi, ndwi):
    manglar = (ndvi > 0.5)
    agua = (ndwi > 0.3)
    return manglar, agua

# 📌 Función corregida para calcular biomasa
def calcular_biomasa(ndvi, ndwi, cobertura_manglar):
    biomasa = np.zeros_like(ndvi)
    biomasa[cobertura_manglar] = 5 + 16 * ndvi[cobertura_manglar] + 30 * ndwi[cobertura_manglar]
    return biomasa

# 📌 Función para calcular carbono y CO₂
def calcular_carbono(biomasa, factor_carbono=0.5):
    return biomasa * factor_carbono

def calcular_co2_equivalente(carbono, factor_co2=3.67):
    return carbono * factor_co2

# 📌 Función para calcular área en hectáreas
def calcular_area_hectareas(mascara):
    area_m2 = np.sum(mascara) * (PIXEL_SIZE ** 2)
    return area_m2 / HECTARE_CONVERSION

# 📌 Función para procesar imágenes
def process_image(bandas, clave):
    required_bands = ['B08', 'B04', 'B11', 'B03']
    if not all(banda in bandas for banda in required_bands):
        print(f"⚠️ Faltan bandas en el conjunto: {bandas.keys()} ({clave})")
        return None, None, None

    try:
        with rasterio.open(bandas['B08']) as src:
            banda_NIR = src.read(1).astype('float32')
        with rasterio.open(bandas['B04']) as src:
            banda_roja = src.read(1).astype('float32')
        with rasterio.open(bandas['B11']) as src:
            banda_SWIR = src.read(1).astype('float32')
        with rasterio.open(bandas['B03']) as src:
            banda_verde = src.read(1).astype('float32')

        # 📌 Crear máscara
        mascara = (banda_roja > 0) & (banda_NIR > 0) & (banda_SWIR > 0) & (banda_verde > 0)
        if np.sum(mascara) == 0:
            print(f"⚠️ Máscara vacía, imagen no válida para: {clave}")
            return None, None, None

        # 📌 Calcular NDVI y NDWI
        ndvi = calcular_ndvi(banda_NIR, banda_roja)
        ndwi = calcular_ndwi(banda_verde, banda_SWIR)

        return ndvi, ndwi, mascara

    except Exception as e:
        print(f"⚠️ Error al procesar {clave}: {e}")
        return None, None, None

# 📌 Procesar imágenes y generar DataFrame
imagenes_anterior = agrupar_imagenes(carpeta_anterior)
imagenes_posterior = agrupar_imagenes(carpeta_posterior)
claves_comunes = imagenes_anterior.keys() & imagenes_posterior.keys()
resultados = []

for clave in claves_comunes:
    bandas_anterior = imagenes_anterior[clave]
    bandas_posterior = imagenes_posterior[clave]

    ndvi_anterior, ndwi_anterior, mascara_anterior = process_image(bandas_anterior, clave)
    ndvi_posterior, ndwi_posterior, mascara_posterior = process_image(bandas_posterior, clave)

    if ndvi_anterior is None or ndvi_posterior is None:
        continue

    manglar_anterior, _ = clasificar_cobertura(ndvi_anterior, ndwi_anterior)
    manglar_posterior, _ = clasificar_cobertura(ndvi_posterior, ndwi_posterior)

    biomasa_anterior = calcular_biomasa(ndvi_anterior, ndwi_anterior, manglar_anterior)
    biomasa_posterior = calcular_biomasa(ndvi_posterior, ndwi_posterior, manglar_posterior)

    area_hectareas_anterior = calcular_area_hectareas(mascara_anterior)
    area_hectareas_posterior = calcular_area_hectareas(mascara_posterior)

    carbono_anterior = calcular_carbono(np.sum(biomasa_anterior))
    carbono_posterior = calcular_carbono(np.sum(biomasa_posterior))
    co2_anterior = calcular_co2_equivalente(carbono_anterior)
    co2_posterior = calcular_co2_equivalente(carbono_posterior)

    depto, shape_leng, zona = clave.split("_", 2)

    # 📌 Extraer el nombre de un archivo de referencia (usamos la banda NIR "B08")
    archivo_referencia = os.path.basename(bandas_posterior['B08']) if 'B08' in bandas_posterior else "Desconocido"

    # Calcular Biomasa por Ha
    biomasa_ha_anterior = np.sum(biomasa_anterior) / area_hectareas_anterior if area_hectareas_anterior > 0 else np.nan
    biomasa_ha_posterior = np.sum(biomasa_posterior) / area_hectareas_posterior if area_hectareas_posterior > 0 else np.nan
    cambio_biomasa_ha = biomasa_ha_posterior - biomasa_ha_anterior

    resultados.append([
        archivo_referencia,
        np.mean(ndvi_anterior), np.mean(ndvi_posterior), np.mean(ndvi_posterior) - np.mean(ndvi_anterior),
        np.sum(biomasa_anterior), np.sum(biomasa_posterior), np.sum(biomasa_posterior) - np.sum(biomasa_anterior),
        area_hectareas_anterior, area_hectareas_posterior, area_hectareas_posterior - area_hectareas_anterior,
        carbono_anterior, carbono_posterior, carbono_posterior - carbono_anterior,
        co2_anterior, co2_posterior, co2_posterior - co2_anterior,
        biomasa_ha_anterior, biomasa_ha_posterior, cambio_biomasa_ha
    ])

# 📌 Crear DataFrame con la estructura correcta
df_resultados = pd.DataFrame(resultados, columns=[
    "Archivo",
    "NDVI Anterior", "NDVI Posterior", "Cambio NDVI",
    "Biomasa Total Anterior (Ton)", "Biomasa Total Posterior (Ton)", "Cambio Biomasa (Ton)",
    "Área (Ha) Anterior", "Área (Ha) Posterior", "Cambio Área (Ha)",
    "Carbono Anterior (Ton)", "Carbono Posterior (Ton)", "Cambio Carbono (Ton)",
    "CO2 Anterior (Ton)", "CO2 Posterior (Ton)", "Cambio CO2 (Ton)",
    "Biomasa por Ha Anterior (Ton/Ha)", "Biomasa por Ha Posterior (Ton/Ha)", "Cambio Biomasa por Ha (Ton/Ha)"
])


df_resultados[['Shape_Leng', 'Fecha', 'Zona']] = df_resultados['Archivo'].str.extract(

    r'_(\d+\.\d+)_([\d]{4}-[\d]{2}-[\d]{2})_([\w-]+)_sentinel-2'
)


df_resultados['Shape_Leng'] = pd.to_numeric(df_resultados['Shape_Leng'], errors='coerce')
df_resultados['Zona'] = df_resultados['Zona'].fillna("Desconocido")
#df_resultados['Departamento'] = df_resultados['Departamento'].fillna("Desconocido")


# Convertir Shape_Leng a tipo numérico y manejar valores faltantes
df_resultados['Shape_Leng'] = pd.to_numeric(df_resultados['Shape_Leng'], errors='coerce').fillna(-1)
df_resultados['Zona'] = df_resultados['Zona'].fillna("Desconocido")

# Identificar y mostrar archivos problemáticos
filas_problematicas = df_resultados[df_resultados[['Shape_Leng', 'Zona']].isnull().any(axis=1)]
if not filas_problematicas.empty:
    print("Archivos con problemas:")
    print(filas_problematicas[['Archivo']])
    filas_problematicas.to_csv("archivos_problematicos.csv", index=False, encoding="utf-8")



# Convertir 'Shape_Leng' a tipo numérico
df_resultados['Shape_Leng'] = pd.to_numeric(df_resultados['Shape_Leng'], errors='coerce')

# Reordenar las columnas para que 'Shape_Leng' y 'Zona' estén al comienzo
columnas_ordenadas = ['Shape_Leng', 'Zona'] + [col for col in df_resultados.columns if col not in ['Shape_Leng', 'Zona']]
df_resultados = df_resultados[columnas_ordenadas]

print(df_resultados[['Shape_Leng', 'Fecha', 'Zona']].head(10))

⚠️ Máscara vacía, imagen no válida para: choco_22.59902627_bajo_baudo
⚠️ Máscara vacía, imagen no válida para: choco_22.59902627_bajo_baudo
⚠️ Máscara vacía, imagen no válida para: choco_1.52376584746_bajo_baudo
⚠️ Máscara vacía, imagen no válida para: choco_1.52376584746_bajo_baudo
⚠️ Máscara vacía, imagen no válida para: choco_22.1558809127_bajo_baudo
⚠️ Máscara vacía, imagen no válida para: choco_22.1558809127_bajo_baudo
⚠️ Máscara vacía, imagen no válida para: choco_7.20204237162_bajo_baudo
⚠️ Máscara vacía, imagen no válida para: choco_7.20204237162_bajo_baudo
⚠️ Máscara vacía, imagen no válida para: choco_44.9567897381_el_litoral_del_san_juan
⚠️ Máscara vacía, imagen no válida para: choco_44.9567897381_el_litoral_del_san_juan
⚠️ Máscara vacía, imagen no válida para: choco_25.937374229_bajo_baudo
⚠️ Máscara vacía, imagen no válida para: choco_25.937374229_bajo_baudo
⚠️ Máscara vacía, imagen no válida para: choco_6.65076698276_el_litoral_del_san_juan
⚠️ Máscara vacía, imagen no vál

In [28]:
df_resultados

,Shape_Leng,Zona,Archivo,NDVI Anterior,NDVI Posterior,Cambio NDVI,Biomasa Total Anterior (Ton),Biomasa Total Posterior (Ton),Cambio Biomasa (Ton),Área (Ha) Anterior,...,Carbono Anterior (Ton),Carbono Posterior (Ton),Cambio Carbono (Ton),CO2 Anterior (Ton),CO2 Posterior (Ton),Cambio CO2 (Ton),Biomasa por Ha Anterior (Ton/Ha),Biomasa por Ha Posterior (Ton/Ha),Cambio Biomasa por Ha (Ton/Ha),Fecha
0,441.919244,El_Litoral_Del_San_Juan,Choco_441.919244473_2024-02-25_El_Litoral_Del_...,-0.000537,0.082388,0.082925,154.769684,205.633179,50.863495,0.90,...,77.384842,102.816589,25.431747,284.002370,377.336883,93.334513,171.966315,228.481310,56.514994,2024-02-25
1,390.567019,Bajo_Baudo,Choco_390.567018841_2025-02-19_Bajo_Baudo_sent...,0.095072,0.098901,0.003829,154.819611,127.427376,-27.392235,0.24,...,77.409805,63.713688,-13.696117,284.093985,233.829235,-50.264751,645.081711,530.947399,-114.134312,2025-02-19
2,123.424807,El_Litoral_Del_San_Juan,Choco_123.424807349_2024-02-25_El_Litoral_Del_...,0.237462,0.231117,-0.006345,4.866179,11.440047,6.573869,0.08,...,2.433089,5.720024,3.286934,8.929438,20.992487,12.063049,60.827231,143.000591,82.173359,2024-02-25
3,37.658465,El_Litoral_Del_San_Juan,Choco_37.6584653544_2024-02-25_El_Litoral_Del_...,0.286550,0.284887,-0.001663,-0.261869,1.595840,1.857710,0.01,...,-0.130935,0.797920,0.928855,-0.480530,2.928367,3.408898,-26.186943,159.584045,185.770988,2024-02-25
4,2172.662027,El_Litoral_Del_San_Juan,Choco_2172.6620274_2024-02-25_El_Litoral_Del_S...,0.423684,0.413964,-0.009720,5692.593262,5902.463867,209.870605,15.24,...,2846.296631,2951.231934,104.935303,10445.908635,10831.021196,385.112561,373.529742,387.300779,13.771037,2024-02-25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273,853.020085,Bajo_Baudo,Choco_853.020084832_2025-02-19_Bajo_Baudo_sent...,0.472444,0.472660,0.000216,643.322998,613.098633,-30.224365,3.11,...,321.661499,306.549316,-15.112183,1180.497701,1125.035991,-55.461710,206.856269,197.137824,-9.718445,2025-02-19
274,767.464719,Bajo_Baudo,Choco_767.464718591_2025-02-19_Bajo_Baudo_sent...,0.342268,0.342195,-0.000073,609.572754,582.358093,-27.214661,1.82,...,304.786377,291.179047,-13.607330,1118.566003,1068.627101,-49.938902,334.930085,319.976974,-14.953110,2025-02-19
275,158.080909,El_Litoral_Del_San_Juan,Choco_158.080908698_2024-02-25_El_Litoral_Del_...,0.224385,0.216458,-0.007927,28.287802,13.378463,-14.909339,0.09,...,14.143901,6.689231,-7.454669,51.908116,24.549479,-27.358637,314.308908,148.649587,-165.659322,2024-02-25
276,142.996631,Bajo_Baudo,Choco_142.996630699_2025-02-19_Bajo_Baudo_sent...,0.064144,0.092826,0.028682,15.702349,29.280811,13.578463,0.04,...,7.851174,14.640406,6.789231,28.813810,53.730289,24.916479,392.558718,732.020283,339.461565,2025-02-19


In [27]:
# @title Descargar el csv de los resultados
from google.colab import files


df_resultados.to_csv('df_resultados.csv', encoding='utf-8')
files.download('df_resultados.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [29]:
# @title Agrega los datos de los analisis al DF
import pandas as pd

# Asegurar que las claves coincidan en ambos DataFrames
dfurl["Shape_Leng"] = dfurl["Shape_Leng"].astype(str).str.strip()
df_resultados["Shape_Leng"] = df_resultados["Shape_Leng"].astype(str).str.strip()


dfurl["Shape_Leng"] = pd.to_numeric(dfurl["Shape_Leng"], errors='coerce')
df_resultados["Shape_Leng"] = pd.to_numeric(df_resultados["Shape_Leng"], errors='coerce')


# Fusionar los DataFrames en base a Departamento, Shape_Leng y Zona
df_final = df.merge(df_resultados,
                    left_on=["Shape_Leng"],
                    right_on=["Shape_Leng"],
                    how="left")

# Mantener `URL_GeoJSON` de `dfurl` en `df_final`
#if "URL_GeoJSON" in dfurl.columns:
#    df_final = df.merge(dfurl[["Archivo", "URL_GeoJSON"]], on="Archivo", how="left")

# preuba sugerida por gemini
if "URL_GeoJSON" in dfurl.columns:
    # Here's the fix: Merge using 'df' instead of 'df_final', as 'df' still has 'Archivo'
    df_final = df.merge(dfurl[["Archivo", "URL_GeoJSON"]], on="Archivo", how="left").merge(df_resultados, left_on=["Shape_Leng"], right_on=["Shape_Leng"], how="left")
    # Alternatively, you can merge using dfurl directly since it's a copy of df with URL_GeoJSON.
    # df_final = dfurl.merge(df_resultados, left_on=["Shape_Leng"], right_on=["Shape_Leng"], how="left")



In [31]:
df_final

,Depto,Restauraci,zona,Shape_Leng,Shape_Area,Area_ha,Archivo_x,Poligono,Coordenadas,Latitud_1,...,Carbono Anterior (Ton),Carbono Posterior (Ton),Cambio Carbono (Ton),CO2 Anterior (Ton),CO2 Posterior (Ton),Cambio CO2 (Ton),Biomasa por Ha Anterior (Ton/Ha),Biomasa por Ha Posterior (Ton/Ha),Cambio Biomasa por Ha (Ton/Ha),Fecha
0,Narino,Restauracion Activa,Narino,560.392623,11925.924601,1.192592,Narino_560.392623449.geojson,"[[-78.61743003297217, 2.310687846041306], [-78...","2.310687846041306, -78.61743003297217",2.310688,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Narino,Restauracion Activa,Narino,1538.249271,47932.706057,4.793271,Narino_1538.24927121.geojson,"[[-78.6355105759848, 2.309385949118805], [-78....","2.309385949118805, -78.6355105759848",2.309386,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Narino,Restauracion Activa,Narino,676.423474,23141.466229,2.314147,Narino_676.423473557.geojson,"[[-78.62558918685171, 2.310687290403761], [-78...","2.310687290403761, -78.62558918685171",2.310687,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Narino,Restauracion Activa,Narino,1731.795248,32024.531823,3.202453,Narino_1731.79524804.geojson,"[[-78.61327318296796, 2.320396349695263], [-78...","2.320396349695263, -78.61327318296796",2.320396,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Narino,Restauracion Activa,Narino,793.963864,10664.562951,1.066456,Narino_793.963864221.geojson,"[[-78.61427941518029, 2.310985757377912], [-78...","2.310985757377912, -78.61427941518029",2.310986,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2071,Choco,Restauracion Activa,Zona 2 - Choco (Bajo Baudo),181.725479,691.657591,0.069166,Choco_181.725479264.geojson,"[[-77.3085641942588, 4.809580030392463], [-77....","4.809580030392463, -77.3085641942588",4.809580,...,12.267122,0.000000,-12.267122,45.020339,0.000000,-45.020339,490.684891,0.000000,-490.684891,2025-02-19
2072,Choco,Restauracion Activa,Zona 2 - Choco (Bajo Baudo),589.301863,17880.350574,1.788035,Choco_589.301862777.geojson,"[[-77.2897446030988, 4.807926009288338], [-77....","4.807926009288338, -77.2897446030988",4.807926,...,364.250977,196.164185,-168.086792,1336.801084,719.922557,-616.878527,402.487267,216.756005,-185.731262,2025-02-19
2073,Choco,Restauracion Activa,Zona 2 - Choco (Bajo Baudo),51.284200,80.904360,0.008090,Choco_51.2841997378.geojson,"[[-77.30807190167423, 4.808746153048969], [-77...","4.808746153048969, -77.30807190167423",4.808746,...,1.121826,0.000000,-1.121826,4.117100,0.000000,-4.117100,224.365139,0.000000,-224.365139,2025-02-19
2074,Choco,Restauracion Activa,Zona 2 - Choco (Bajo Baudo),10.641806,3.916600,0.000392,Choco_10.6418057464.geojson,"[[-77.3083871092495, 4.80886239956291], [-77.3...","4.80886239956291, -77.3083871092495",4.808862,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
# @title ETL DataFrame
# Eliminar columnas
df_final = df_final.drop(columns=['zona', 'Archivo_y'], errors='ignore')

# Renombrar la columna Zona
df_final = df_final.rename(columns={'Zona': 'municipio'})

df_final


,Depto,Restauraci,Shape_Leng,Shape_Area,Area_ha,Archivo_x,Poligono,Coordenadas,Latitud_1,Longitud_1,...,Carbono Anterior (Ton),Carbono Posterior (Ton),Cambio Carbono (Ton),CO2 Anterior (Ton),CO2 Posterior (Ton),Cambio CO2 (Ton),Biomasa por Ha Anterior (Ton/Ha),Biomasa por Ha Posterior (Ton/Ha),Cambio Biomasa por Ha (Ton/Ha),Fecha
0,Narino,Restauracion Activa,560.392623,11925.924601,1.192592,Narino_560.392623449.geojson,"[[-78.61743003297217, 2.310687846041306], [-78...","2.310687846041306, -78.61743003297217",2.310688,-78.617430,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Narino,Restauracion Activa,1538.249271,47932.706057,4.793271,Narino_1538.24927121.geojson,"[[-78.6355105759848, 2.309385949118805], [-78....","2.309385949118805, -78.6355105759848",2.309386,-78.635511,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Narino,Restauracion Activa,676.423474,23141.466229,2.314147,Narino_676.423473557.geojson,"[[-78.62558918685171, 2.310687290403761], [-78...","2.310687290403761, -78.62558918685171",2.310687,-78.625589,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Narino,Restauracion Activa,1731.795248,32024.531823,3.202453,Narino_1731.79524804.geojson,"[[-78.61327318296796, 2.320396349695263], [-78...","2.320396349695263, -78.61327318296796",2.320396,-78.613273,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Narino,Restauracion Activa,793.963864,10664.562951,1.066456,Narino_793.963864221.geojson,"[[-78.61427941518029, 2.310985757377912], [-78...","2.310985757377912, -78.61427941518029",2.310986,-78.614279,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2071,Choco,Restauracion Activa,181.725479,691.657591,0.069166,Choco_181.725479264.geojson,"[[-77.3085641942588, 4.809580030392463], [-77....","4.809580030392463, -77.3085641942588",4.809580,-77.308564,...,12.267122,0.000000,-12.267122,45.020339,0.000000,-45.020339,490.684891,0.000000,-490.684891,2025-02-19
2072,Choco,Restauracion Activa,589.301863,17880.350574,1.788035,Choco_589.301862777.geojson,"[[-77.2897446030988, 4.807926009288338], [-77....","4.807926009288338, -77.2897446030988",4.807926,-77.289745,...,364.250977,196.164185,-168.086792,1336.801084,719.922557,-616.878527,402.487267,216.756005,-185.731262,2025-02-19
2073,Choco,Restauracion Activa,51.284200,80.904360,0.008090,Choco_51.2841997378.geojson,"[[-77.30807190167423, 4.808746153048969], [-77...","4.808746153048969, -77.30807190167423",4.808746,-77.308072,...,1.121826,0.000000,-1.121826,4.117100,0.000000,-4.117100,224.365139,0.000000,-224.365139,2025-02-19
2074,Choco,Restauracion Activa,10.641806,3.916600,0.000392,Choco_10.6418057464.geojson,"[[-77.3083871092495, 4.80886239956291], [-77.3...","4.80886239956291, -77.3083871092495",4.808862,-77.308387,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Url Imágenes con las bandas

In [33]:
# @title Generar URL para las imagenes de las bandas
import os
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import pandas as pd
import re

# Autenticación en Google Drive
auth.authenticate_user()
drive_service = build('drive', 'v3')

# Nombre de la carpeta donde están las imágenes
folder_name = "pacifico_posterior_completo"

# Obtener el ID de la carpeta de Google Drive
def get_folder_id(folder_name):
    try:
        results = drive_service.files().list(
            q=f"mimeType='application/vnd.google-apps.folder' and name='{folder_name}'",
            spaces="drive",
            fields="files(id, name)"
        ).execute()
        items = results.get('files', [])
        return items[0]['id'] if items else None
    except HttpError as error:
        print(f'Error al obtener el ID de la carpeta: {error}')
        return None

# Obtener la lista de imágenes dentro de la carpeta
def get_images_in_folder(folder_id):
    try:
        image_files = []
        page_token = None
        query = f"'{folder_id}' in parents and (mimeType='image/jpeg' or mimeType='image/png' or mimeType='image/tiff')"

        while True:
            results = drive_service.files().list(
                q=query,
                fields="nextPageToken, files(id, name, webViewLink)",
                pageSize=1000,  # Máximo permitido en Drive
                pageToken=page_token
            ).execute()

            image_files.extend(results.get('files', []))
            page_token = results.get('nextPageToken')
            if not page_token:
                break  # Salir si no hay más páginas

        return image_files
    except HttpError as error:
        print(f'Error al obtener las imágenes: {error}')
        return []

# Obtener el ID de la carpeta
folder_id = get_folder_id(folder_name)

if folder_id:
    # Obtener las imágenes dentro de la carpeta
    image_files = get_images_in_folder(folder_id)

    # Crear un diccionario para agrupar las imágenes por clave única
    image_dict = {}

    for image in image_files:
        file_name = image['name']
        file_id = image['id']
        file_url = f'https://drive.google.com/file/d/{file_id}/view'

        # Extraer la clave única del archivo (Departamento, Shape_Leng, Año, Zona)
        match = re.match(r"^(.+?)_(\d+\.\d+?)_(\d{4})_(.+?)_sentinel-2_(B\d+)\.tiff$", file_name)

        if match:
            depto, shape_leng, year, zona, banda = match.groups()
            key = f"{depto}_{shape_leng}_{year}_{zona}"  # Clave única sin la banda

            # Asegurar que la clave existe en el diccionario
            if key not in image_dict:
                image_dict[key] = {"Shape_Leng": shape_leng, "Banda_11": "", "Banda_4": "", "Banda_3": "", "Banda_8": ""}

            # Asignar la URL a la banda correspondiente
            if banda in ["B011", "B11"]:
                image_dict[key]["Banda_11"] = file_url
            elif banda == "B04":
                image_dict[key]["Banda_4"] = file_url
            elif banda == "B03":
                image_dict[key]["Banda_3"] = file_url
            elif banda == "B08":
                image_dict[key]["Banda_8"] = file_url

    # Convertir el diccionario en un DataFrame
    df_bandas = pd.DataFrame.from_dict(image_dict, orient="index").reset_index()
    df_bandas.columns = ["Clave_Unica", "Shape_Leng", "Banda_11", "Banda_4", "Banda_3", "Banda_8"]

    # Convertir `Shape_Leng` a tipo numérico
    df_bandas["Shape_Leng"] = pd.to_numeric(df_bandas["Shape_Leng"], errors="coerce")



else:
    print("❌ No se pudo encontrar la carpeta en Google Drive.")
df_bandas

,Clave_Unica,Shape_Leng,Banda_11,Banda_4,Banda_3,Banda_8
0,Narino_603.846122512_2024_tumaco_zona1,603.846123,https://drive.google.com/file/d/1IV54KnfSaenZE...,https://drive.google.com/file/d/1Y7i6XTxC2XBxm...,https://drive.google.com/file/d/1bcxc_U-j_LNxl...,https://drive.google.com/file/d/1dSYDjDgkF7E_7...
1,Narino_726.758597965_2024_tumaco_zona1,726.758598,https://drive.google.com/file/d/1oUYcZI7I6sxNQ...,https://drive.google.com/file/d/1K0CE-Wo-SjuYe...,https://drive.google.com/file/d/1grzyQMhmdy72u...,https://drive.google.com/file/d/1l2gJs79XDbJ5w...
2,Narino_629.486215032_2024_tumaco_zona1,629.486215,https://drive.google.com/file/d/1dHbQGeHeOHWzc...,https://drive.google.com/file/d/1FmOH6o4jpTT_u...,https://drive.google.com/file/d/1kyCqZFlmrzaVQ...,https://drive.google.com/file/d/1yn3_s23Rll8GE...
3,Narino_579.975542111_2024_tumaco_zona1,579.975542,https://drive.google.com/file/d/1OJTMtIXuQBdKw...,https://drive.google.com/file/d/1Wbd23qivPDsSF...,https://drive.google.com/file/d/1X6iBjavr-4O1n...,https://drive.google.com/file/d/1A6ma5mfNiSZCk...
4,Narino_591.552765048_2024_tumaco_zona1,591.552765,https://drive.google.com/file/d/1Gm22x39Y6Tqp7...,https://drive.google.com/file/d/1jn-RMectORR5-...,https://drive.google.com/file/d/1-GmWXx9ocNYvB...,https://drive.google.com/file/d/1CF06-5-_eVd6t...
...,...,...,...,...,...,...
2071,Choco_22.59902627_2024_bajo_baudo,22.599026,https://drive.google.com/file/d/1_wMuNUURqXUwc...,https://drive.google.com/file/d/11PjbSBC1VMoVc...,https://drive.google.com/file/d/1yy0-w8uLd6JIi...,https://drive.google.com/file/d/1VPT0_InPowgAx...
2072,Choco_375.388428572_2024_bajo_baudo,375.388429,https://drive.google.com/file/d/12wA3WKvMFPAL8...,https://drive.google.com/file/d/1Xby_3XWbFzKhA...,https://drive.google.com/file/d/1lsRAnCsPIT4Qq...,https://drive.google.com/file/d/1BH51K-VRfasYL...
2073,Choco_32.6532003516_2024_bajo_baudo,32.653200,https://drive.google.com/file/d/1pGcY_KXZK0Iot...,https://drive.google.com/file/d/144RnDG7mhf4QV...,https://drive.google.com/file/d/1uOTvfdMYTL2sB...,https://drive.google.com/file/d/1ow85HkekYeffz...
2074,Choco_394.717999907_2024_bajo_baudo,394.718000,https://drive.google.com/file/d/1cZN9dRny1eKKh...,https://drive.google.com/file/d/1GIGh-OfLwTG2c...,https://drive.google.com/file/d/1TmNqaGeyB0W5X...,https://drive.google.com/file/d/1xjKO6EZ_KJ2F6...


In [34]:
# @title Union df_bandas con DataFrame principal
import pandas as pd

# 📌 Asegurar que Shape_Leng es numérico en ambos DataFrames
df_final["Shape_Leng"] = pd.to_numeric(df_final["Shape_Leng"], errors="coerce")
df_bandas["Shape_Leng"] = pd.to_numeric(df_bandas["Shape_Leng"], errors="coerce")

# 📌 Fusionar los DataFrames usando Shape_Leng
df_final = df_final.merge(df_bandas, on="Shape_Leng", how="left")

In [35]:
df_final

,Depto,Restauraci,Shape_Leng,Shape_Area,Area_ha,Archivo_x,Poligono,Coordenadas,Latitud_1,Longitud_1,...,Cambio CO2 (Ton),Biomasa por Ha Anterior (Ton/Ha),Biomasa por Ha Posterior (Ton/Ha),Cambio Biomasa por Ha (Ton/Ha),Fecha,Clave_Unica,Banda_11,Banda_4,Banda_3,Banda_8
0,Narino,Restauracion Activa,560.392623,11925.924601,1.192592,Narino_560.392623449.geojson,"[[-78.61743003297217, 2.310687846041306], [-78...","2.310687846041306, -78.61743003297217",2.310688,-78.617430,...,NaN,NaN,NaN,NaN,NaN,Narino_560.392623449_2024_zona2,https://drive.google.com/file/d/1atIF9k3QhRfZY...,https://drive.google.com/file/d/1rPDzG35ppCqTD...,https://drive.google.com/file/d/1OIMHFXV9hPmOu...,https://drive.google.com/file/d/15j8_RflrBXXzT...
1,Narino,Restauracion Activa,1538.249271,47932.706057,4.793271,Narino_1538.24927121.geojson,"[[-78.6355105759848, 2.309385949118805], [-78....","2.309385949118805, -78.6355105759848",2.309386,-78.635511,...,NaN,NaN,NaN,NaN,NaN,Narino_1538.24927121_2024_zona2,https://drive.google.com/file/d/1_GgsNRY__NFUV...,https://drive.google.com/file/d/1KXB7PoOYLt2rv...,https://drive.google.com/file/d/16P0rgI_C0B-iH...,https://drive.google.com/file/d/18EYXgXy5zIChs...
2,Narino,Restauracion Activa,676.423474,23141.466229,2.314147,Narino_676.423473557.geojson,"[[-78.62558918685171, 2.310687290403761], [-78...","2.310687290403761, -78.62558918685171",2.310687,-78.625589,...,NaN,NaN,NaN,NaN,NaN,Narino_676.423473557_2024_zona2,https://drive.google.com/file/d/1ti4cIRLgvQwLs...,https://drive.google.com/file/d/1emNRzGpTR23Wa...,https://drive.google.com/file/d/1N6tC-zDyQUFb5...,https://drive.google.com/file/d/1j5p_F4vEppMkq...
3,Narino,Restauracion Activa,1731.795248,32024.531823,3.202453,Narino_1731.79524804.geojson,"[[-78.61327318296796, 2.320396349695263], [-78...","2.320396349695263, -78.61327318296796",2.320396,-78.613273,...,NaN,NaN,NaN,NaN,NaN,Narino_1731.79524804_2024_zona2,https://drive.google.com/file/d/1GbQZxUlMU19_B...,https://drive.google.com/file/d/1_RGoSBvDKEe41...,https://drive.google.com/file/d/1SbjWSOgEvWQyp...,https://drive.google.com/file/d/1XGjp41fmJExrw...
4,Narino,Restauracion Activa,793.963864,10664.562951,1.066456,Narino_793.963864221.geojson,"[[-78.61427941518029, 2.310985757377912], [-78...","2.310985757377912, -78.61427941518029",2.310986,-78.614279,...,NaN,NaN,NaN,NaN,NaN,Narino_793.963864221_2024_zona2,https://drive.google.com/file/d/1K-_Cft3_H1gaF...,https://drive.google.com/file/d/1RAkhqOq4V9Fai...,https://drive.google.com/file/d/1V4gOK-bb8dcqz...,https://drive.google.com/file/d/1u51--IFRICLR9...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2071,Choco,Restauracion Activa,181.725479,691.657591,0.069166,Choco_181.725479264.geojson,"[[-77.3085641942588, 4.809580030392463], [-77....","4.809580030392463, -77.3085641942588",4.809580,-77.308564,...,-45.020339,490.684891,0.000000,-490.684891,2025-02-19,Choco_181.725479264_2024_bajo_baudo,https://drive.google.com/file/d/1TDiAMCtYle2fm...,https://drive.google.com/file/d/1mulgLSXfWVQuJ...,https://drive.google.com/file/d/1nJ0C2gW1RaUMN...,https://drive.google.com/file/d/14TM4WhfzqqGvk...
2072,Choco,Restauracion Activa,589.301863,17880.350574,1.788035,Choco_589.301862777.geojson,"[[-77.2897446030988, 4.807926009288338], [-77....","4.807926009288338, -77.2897446030988",4.807926,-77.289745,...,-616.878527,402.487267,216.756005,-185.731262,2025-02-19,Choco_589.301862777_2024_bajo_baudo,https://drive.google.com/file/d/1RHYoKaqX-CpYc...,https://drive.google.com/file/d/118Dcd2GfAueQK...,https://drive.google.com/file/d/1PPlUhV0lwN6Y-...,https://drive.google.com/file/d/1G33CusVFdCrcg...
2073,Choco,Restauracion Activa,51.284200,80.904360,0.008090,Choco_51.2841997378.geojson,"[[-77.30807190167423, 4.808746153048969], [-77...","4.808746153048969, -77.30807190167423",4.808746,-77.308072,...,-4.117100,224.365139,0.000000,-224.365139,2025-02-19,Choco_51.2841997378_2024_bajo_baudo,https://drive.google.com/file/d/1nAOJD1npuDC34...,https://

In [36]:
# @title descarga el df_final
from google.colab import files

df_final.to_csv('df_final.csv', encoding='utf-8')
files.download('df_final.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Entrenamiento para predicción

In [37]:
df = df_final.copy()

In [39]:
# @title Calculo a 10 años Predicción
# Cargar librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score



# Seleccionar variables de interés
features = ['NDVI Anterior', 'NDVI Posterior', 'Área (Ha) Anterior', 'Área (Ha) Posterior', 'Biomasa Total Anterior (Ton)', 'Biomasa Total Posterior (Ton)']

# Convertir las columnas a valores numéricos, reemplazando errores
for col in features:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Filtrar datos limpios
df_clean = df.dropna(subset=features)

# Predicción de Biomasa
X_biomasa = df_clean[['NDVI Anterior', 'NDVI Posterior', 'Área (Ha) Anterior', 'Área (Ha) Posterior', 'Biomasa Total Anterior (Ton)']]
y_biomasa = df_clean['Biomasa Total Posterior (Ton)']
X_train_b, X_test_b, y_train_b, y_test_b = train_test_split(X_biomasa, y_biomasa, test_size=0.2, random_state=42)

# Modelo de regresión lineal para Biomasa
linear_model_b = LinearRegression()
linear_model_b.fit(X_train_b, y_train_b)
df_clean['Predicción Biomasa Regresión Lineal 10 años'] = linear_model_b.predict(X_biomasa)

# Modelo Random Forest para Biomasa
rf_model_b = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model_b.fit(X_train_b, y_train_b)
df_clean['Predicción Biomasa Random Forest 10 años'] = rf_model_b.predict(X_biomasa)

# Predicción de NDVI
X_ndvi = df_clean[['NDVI Anterior', 'Área (Ha) Anterior', 'Área (Ha) Posterior', 'Biomasa Total Anterior (Ton)', 'Biomasa Total Posterior (Ton)']]
y_ndvi = df_clean['NDVI Posterior']
X_train_n, X_test_n, y_train_n, y_test_n = train_test_split(X_ndvi, y_ndvi, test_size=0.2, random_state=42)

# Modelo de regresión lineal para NDVI
linear_model_n = LinearRegression()
linear_model_n.fit(X_train_n, y_train_n)
df_clean['Predicción NDVI Regresión Lineal 10 años'] = linear_model_n.predict(X_ndvi)

# Modelo Random Forest para NDVI
rf_model_n = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model_n.fit(X_train_n, y_train_n)
df_clean['Predicción NDVI Random Forest 10 años'] = rf_model_n.predict(X_ndvi)

# Evaluar modelos
def evaluar_modelo(y_true, y_pred, modelo):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    print(f'\nResultados de {modelo}:')
    print(f'MAE: {mae:.2f}')
    print(f'MSE: {mse:.2f}')
    print(f'RMSE: {rmse:.2f}')
    print(f'R2: {r2:.2f}')

# Mostrar resultados
evaluar_modelo(y_test_b, linear_model_b.predict(X_test_b), "Regresión Lineal Biomasa")
evaluar_modelo(y_test_b, rf_model_b.predict(X_test_b), "Random Forest Biomasa")
evaluar_modelo(y_test_n, linear_model_n.predict(X_test_n), "Regresión Lineal NDVI")
evaluar_modelo(y_test_n, rf_model_n.predict(X_test_n), "Random Forest NDVI")

df_clean


# Guardar el DataFrame con predicciones
#df_clean.to_csv('/content/drive/MyDrive/df_final_con_predicciones.csv', index=False)
#print("Archivo guardado: df_final_con_predicciones.csv")

<ipython-input-39-369cdc4f7439>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['Predicción Biomasa Regresión Lineal 10 años'] = linear_model_b.predict(X_biomasa)
<ipython-input-39-369cdc4f7439>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['Predicción Biomasa Random Forest 10 años'] = rf_model_b.predict(X_biomasa)
<ipython-input-39-369cdc4f7439>:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v


Resultados de Regresión Lineal Biomasa:
MAE: 57.42
MSE: 8669.82
RMSE: 93.11
R2: 0.90

Resultados de Random Forest Biomasa:
MAE: 67.86
MSE: 13845.76
RMSE: 117.67
R2: 0.85

Resultados de Regresión Lineal NDVI:
MAE: 0.03
MSE: 0.00
RMSE: 0.05
R2: 0.87

Resultados de Random Forest NDVI:
MAE: 0.03
MSE: 0.00
RMSE: 0.04
R2: 0.91


<ipython-input-39-369cdc4f7439>:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['Predicción NDVI Random Forest 10 años'] = rf_model_n.predict(X_ndvi)


,Depto,Restauraci,Shape_Leng,Shape_Area,Area_ha,Archivo_x,Poligono,Coordenadas,Latitud_1,Longitud_1,...,Fecha,Clave_Unica,Banda_11,Banda_4,Banda_3,Banda_8,Predicción Biomasa Regresión Lineal 10 años,Predicción Biomasa Random Forest 10 años,Predicción NDVI Regresión Lineal 10 años,Predicción NDVI Random Forest 10 años
925,Choco,Manejo Forestal Sostenible e Incentivos a la C...,371.493501,6687.928979,0.668793,Choco_371.493501341.geojson,"[[-77.52460565226096, 4.213008978334686], [-77...","4.213008978334686, -77.52460565226096",4.213009,-77.524606,...,2024-02-25,Choco_371.493501341_2024_litoral_san_juan,https://drive.google.com/file/d/1mvQMKQlbDnwv5...,https://drive.google.com/file/d/1YpgGriAXGOJn-...,https://drive.google.com/file/d/1o7evzY6qWiUiy...,https://drive.google.com/file/d/1aCqb90yOp36v-...,258.696646,262.257409,0.361703,0.381851
926,Choco,Manejo Forestal Sostenible e Incentivos a la C...,37.658465,15.989146,0.001599,Choco_37.6584653544.geojson,"[[-77.52547128761086, 4.22004092415558], [-77....","4.22004092415558, -77.52547128761086",4.220041,-77.525471,...,2024-02-25,Choco_37.6584653544_2024_litoral_san_juan,https://drive.google.com/file/d/1_4DJTgLAP5Cqv...,https://drive.google.com/file/d/1fvDTTTeR4WzFf...,https://drive.google.com/file/d/1-4D8uKMjY5AxR...,https://drive.google.com/file/d/1J_RXSV2C1L1Zj...,-27.413863,2.217830,0.271658,0.282900
927,Choco,Manejo Forestal Sostenible e Incentivos a la C...,40.675799,9.022308,0.000902,Choco_40.6757985674.geojson,"[[-77.52640737905283, 4.221428485210787], [-77...","4.221428485210787, -77.52640737905283",4.221428,-77.526407,...,2024-02-25,Choco_40.6757985674_2024_litoral_san_juan,https://drive.google.com/file/d/1aJal0Qjk2NZnB...,https://drive.google.com/file/d/1DlYx3ORTjtPnk...,https://drive.google.com/file/d/1nOuL99h1YHvdb...,https://drive.google.com/file/d/1Urm7i62gIm_-c...,-13.599185,3.397378,0.242452,0.250474
928,Choco,Manejo Forestal Sostenible e Incentivos a la C...,1005.015109,26050.101673,2.605010,Choco_1005.01510876.geojson,"[[-77.4751097102653, 4.219925517572741], [-77....","4.219925517572741, -77.4751097102653",4.219926,-77.475110,...,2024-02-25,Choco_1005.01510876_2024_litoral_san_juan,https://drive.google.com/file/d/18Ka53mSuv6DLN...,https://drive.google.com/file/d/1QRxxbW6AqjNqP...,https://drive.google.com/file/d/1GTW-pcS6mxd8m...,https://drive.google.com/file/d/1HYWiXVpQzjEaP...,1039.497128,812.920299,0.237162,0.228114
929,Choco,Manejo Forestal Sostenible e Incentivos a la C...,3763.296052,809843.835799,80.984384,Choco_3763.29605169.geojson,"[[-77.48794490614327, 4.212576700730082], [-77...","4.212576700730082, -77.48794490614327",4.212577,-77.487945,...,2024-02-25,Choco_3763.29605169_2024_litoral_san_juan,https://drive.google.com/file/d/19idhrMO-ly1_w...,https://drive.google.com/file/d/1-R21T3BDc3mBY...,https://drive.google.com/file/d/1FLyP2nxh2V_eW...,https://drive.google.com/file/d/1xSNUORRkkeIBs...,36550.752048,31525.071357,0.628155,0.536553
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2067,Choco,Restauracion Activa,450.396856,7090.717615,0.709072,Choco_450.396855725.geojson,"[[-77.30738228843339, 4.807463322843386], [-77...","4.807463322843386, -77.30738228843339",4.807463,-77.307382,...,2025-02-19,Choco_450.396855725_2024_bajo_baudo,https://drive.google.com/file/d/1YiVoZxhFT8PfL...,https://drive.google.com/file/d/1BTE9xcwFezZ08...,https://drive.google.com/file/d/1oNuLwpCqNOJVm...,https://drive.google.com/file/d/1zGqaHgTf_FihV...,179.617296,86.856321,0.236490,0.100236
2068,Choco,Restauracion Activa,481.982832,5323.497309,0.532350,Choco_481.982832266.geojson,"[[-77.31767622550919, 4.804330240272989], [-77...","4.804330240272989, -77.31767622550919",4.804330,-77.317676,...,2025-02-19,Choco_481.982832266_2024_bajo_baudo,https://drive.google.com/file/d/1R0MIx1URNbSjo...,https://drive.google.com/file/d/1jsZ79-8WlkeSk...,https://drive.google.com/file/d/1PUa21YG-whcKR...,https://drive.google.com/file/d/1V0kabnTTfb0Fb...,125

# Escritura en un Google Sheet Final

In [ ]:
# @title Escribe en Sheet **Final**
import gspread
from google.colab import auth
from google.auth import default
import pandas as pd
import numpy as np
from gspread_dataframe import set_with_dataframe

# 📌 Autenticación en Google Drive
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# 📌 Nombre de la hoja de Google Sheets donde se guardará el DataFrame
nombre_hoja = "Datos_Finales"

# 📌 Convertir valores no compatibles a formatos válidos
df_clean.replace([np.inf, -np.inf], np.nan, inplace=True)  # Reemplaza `inf` por `NaN`
df_clean.fillna("", inplace=True)  # Sustituye `NaN` por valores vacíos

# 📌 Convertir solo las columnas de texto a `str`, manteniendo los números como están
for col in df_clean.select_dtypes(include=['object']).columns:
    df_clean[col] = df_clean[col].astype(str)

# 📌 Limitar el tamaño máximo de cada celda a 50,000 caracteres
df_clean = df_clean.applymap(lambda x: x[:50000] if isinstance(x, str) else x)

# 📌 Crear o abrir la hoja en Google Sheets
try:
    spreadsheet = gc.open(nombre_hoja)  # Intentar abrir la hoja existente
except gspread.exceptions.SpreadsheetNotFound:
    spreadsheet = gc.create(nombre_hoja)  # Si no existe, crear una nueva

# 📌 Seleccionar la primera hoja, si no existe, crearla
worksheet = spreadsheet.sheet1

# 📌 Limpiar la hoja antes de escribir nuevos datos
worksheet.clear()

# 📌 Escribir el DataFrame en Google Sheets sin perder formato numérico
set_with_dataframe(worksheet, df_clean, include_index=False, include_column_header=True)

print(f"✅ Datos guardados correctamente en Google Sheets: {spreadsheet.url}")
